In [12]:
import pandas as pd
from ase import Atoms, Atom
from ase.io import read, write
from ase.visualize import view
from autoadsorbate import Surface, Fragment
from autoadsorbate.Surf import attach_fragment


In [ ]:
# https://github.com/basf/autoadsorbate#
from autoadsorbate import Surface, Fragment
from ase.io import write

f = Fragment('[C+](c1ccccc1)(c1ccccc1)(c1ccccc1)')
write('tmp.xyz', f.get_conformer(0))
#is the carbo cation sterically protected?

In [ ]:
import ase
from typing import Literal

def read_atoms_object(path: str):
    """reads a atomistic structure file from the  system,
    Args:
        path: string - location on system
    returns:
        ase.Atoms object
    """
    return ase.io.read(path)

def get_sites_from_atoms(atoms: ase.Atoms, mode: Literal['slab', 'particle']):
    """
    Args:
        atoms: ase.Atoms object that can be an "slab" or 'particle'. Determines all surface sites. 
    Returns:
        pandas.DataFrame containing all site information.
    """
    return Surface(atoms, mode=mode).site_df

def get_fragment(SMILES: str):
    """
    Args:
        SMILES: string of smiles that should be placed on surface sites.
    returns:
        ase.Atoms of molecule or molecular fragment, alligned relative to the site in [0,0,0]
    """
    return Fragment(SMILES).get_conformer(0)

def get_ads_slab(slab_atoms, fragment_atoms, site_dict):
    """
    Args:
        slab_atoms: ase.Atoms, atoms of slab that should host the fragment
        fragment_atoms: ase.Atoms, molecular fragment obtained from SMILES
        site_dict: dict, information about the selected site geometry
    returns:
        ase.Atoms of molecule placed on slab
    """

    ads_slab_atoms = attach_fragment(
        atoms = slab_atoms,
        site_dict = site_dict,
        fragment = fragment_atoms,
        n_rotation = 0.,
        height=1.5
        
    )

    return ads_slab_atoms


In [ ]:
# Here is my slab: <path_to_slab>
# it should bind to a top site in the center of the slab
# how does *[O+]=C=O bind?


In [18]:
s.site_df.loc[0].to_dict()

{'coordinates': array([0.        , 0.        , 7.08423447]),
 'connectivity': 1,
 'topology': [9],
 'n_vector': array([-0.00909863, -0.01227215,  0.9998833 ]),
 'h_vector': array([1., 0., 0.]),
 'site_formula': {'Cu': 1}}

In [ ]:
# write('/mnt/c/Users/ef/Desktop/tmp/sites.xyz', [s.view_site(i, return_atoms = True) for i in s.site_df.index.values])

In [19]:
s.site_df

,coordinates,connectivity,topology,n_vector,h_vector,site_formula
0,"[0.0, 0.0, 7.084234471774549]",1,[9],"[-0.00909863165381745, -0.01227215012716785, 0...","[1.0, 0.0, 0.0]",{'Cu': 1}
1,"[0.6381638700208592, 1.1053322464309083, 7.084...",2,"[9, 12]","[-0.0005231742671816635, -0.009764909487711476...","[-0.5000000000000002, -0.8660254037844385, 0.0]",{'Cu': 2}
2,"[1.2763277400417183, 0.0, 7.084234471774548]",2,"[9, 10]","[-0.00951817042797705, -0.020823655013298387, ...","[-1.0, 0.0, 0.0]",{'Cu': 2}
3,"[1.2763277400417183, 0.7368881642872724, 7.084...",3,"[9, 10, 12]","[-0.013328313623848497, -0.012919400320029662,...","[-1.0, 0.0, 0.0]",{'Cu': 3}
4,"[1.2763277400417183, 2.2106644928618175, 7.084...",1,[12],"[-0.0029853952101896084, -0.00487023427327535,...","[1.0, 0.0, 0.0]",{'Cu': 1}
5,"[1.914491610062578, 1.1053322464309083, 7.0842...",2,"[10, 12]","[-0.015682322820486996, -0.00570207774996847, ...","[-0.5000000000000004, 0.8660254037844385, 0.0]",{'Cu': 2}
6,"[1.9144916100625773, 3.315996739292726, 7.0842...",2,"[12, 15]","[-0.00687001851725571, 0.003242731170168389, 0...","[-0.5000000000000001, -0.8660254037844385, 0.0]",{'Cu': 2}
7,"[2.5526554800834367, 0.0, 7.084234471774549]",1,[10],"[-0.0029959062894920744, -0.003317156151590183...","[1.0, 0.0, 0.0]",{'Cu': 1}
8,"[2.5526554800834393, 1.4737763285745449, 7.084...",3,"[10, 12, 13]","[-0.0011596349368931467, -0.001445905668585174...","[0.5000000000000004, -0.8660254037844385, 0.0]",{'Cu': 3}
9,"[2.5526554800834385, 2.210664492861816, 7.0842...",2,"[12, 13]","[-0.016025063388777436, 0.00862123961740766, 0...","[-1.0, 0.0, 0.0]",{'Cu': 2}


In [15]:
from ase.build import fcc111

out_trj = []

slab = fcc111('Cu', [3,3,2], vacuum=5)

out_trj.append(slab.copy())

s = Surface(slab)
f = Fragment('ClC(=O)[O-]')

a = attach_fragment(
    atoms = s.atoms,
    site_dict = s.site_df.loc[0].to_dict(),
    fragment = f.get_conformer(0),
    n_rotation = 0.,
    height=1.5
    
)

out_trj.append(a)

write('/mnt/c/Users/ef/Desktop/tmp/tmp.xyz', out_trj)


/home/ef/venvs/llm_adsorbate/lib/python3.11/site-packages/ase/io/extxyz.py:318: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [5]:
# view(a)

In [8]:
Fragment('ClC(=O)[O-]')

In [66]:
from ase.build import bulk, surface
from ase.io import write
import numpy as np
import random
random.seed(48) 


# Step 1: Start with a binary oxide (e.g., NiO in rocksalt structure)
nio = bulk('NiO', 'rocksalt', a=4.17)

# Step 2: Make a surface slab (e.g., (100) surface, 3 layers)
slab = surface(nio, (1, 0, 0), layers=4)
slab.center(vacuum=10, axis=2)  # add vacuum along z
slab *= [4,4,1]

# Step 3: Replace some Ni with Fe to create mixed-metal oxide slab
symbols = slab.get_chemical_symbols()
for i, sym in enumerate(symbols):
    if sym == "Ni" and np.random.rand() < 0.3:  # randomly swap ~30% Ni for Fe
        symbols[i] = "Fe"


slab.set_chemical_symbols(symbols)

s = Surface(slab)

inds = []
for t in s.site_df.topology.values:
    inds+=t
set(inds)


del_lst = random.sample(inds, 15)

slab = slab[[atom.index for atom in slab if atom.index not in del_lst]]
slab.rattle(stdev=.2)

write("NiFeO_slab.xyz", slab)
print("Saved NiFeO_slab.xyz")

Saved NiFeO_slab.xyz
